In [1]:
import random
from importlib import reload

import torch

from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM

from util import mt

In [2]:
model_name = 'EleutherAI/gpt-neo-125M'
model_name = 'facebook/xglm-7.5B'
model_name = 'facebook/xglm-4.5B'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()
model.eval()

XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): Embedding(256008, 2048, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
        )
        (activation_fn): ReLU()
        (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=2048, out_features=16384, bias=True)
        (fc2): Linear(in_features=16384, out_features=2048, bias=True)
        (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      )
      (1): XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Lin

In [183]:
src = load_dataset("gsarti/flores_101", 'fra')
tgt = load_dataset("gsarti/flores_101", 'eng')

Generating dev split:   0%|          | 0/997 [00:00<?, ? examples/s]

Generating devtest split:   0%|          | 0/1012 [00:00<?, ? examples/s]

Dataset flores_101 downloaded and prepared to /gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/fra/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/eng/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

In [184]:
src_endonym = 'French'
tgt_endonym = 'English'

In [ ]:
inference_point = src['dev'][1] 
prompt_ids = demo_prompt_ids + s_ids(f"{src_endonym}: {inference_point['sentence']}") + [2]  + s_ids(f"{tgt_endonym}:")
prompt_t = torch.LongTensor([prompt_ids]).to(model.device)

In [109]:
inference_point = src['dev'][1] 
prompt_ids = demo_prompt_ids + s_ids(f"{src_endonym}: {inference_point['sentence']} | {tgt_endonym}:")
prompt_t = torch.LongTensor([prompt_ids]).to(model.device)

[1075, 17, 418, 4147]

In [82]:
def s_ids(s):
  return tokenizer(s).input_ids[1:]

In [124]:
def make_demo_prompt(src_endonym, tgt_endonym, src, tgt, k):
  src_points = random.sample(list(src), k=k)
  tgt_points = [ tgt[point['id']-1] for point in src_points ]
  demos = []
  for s, t in zip(src_points, tgt_points):
    demo = s_ids(f"{src_endonym}: {s['sentence']}")
    demo += [2]
    demo += s_ids(f"{tgt_endonym}: {t['sentence']}")
    demos.append(demo)
  demo_prompt_ids = [2]
  for demo in demos:
    demo_prompt_ids.extend(demo)
    demo_prompt_ids.append(2)
    demo_prompt_ids.append(2)
  return demo_prompt_ids

In [138]:
def make_demo_prompt(src_endonym, tgt_endonym, src, tgt, k):
  src_points = random.sample(list(src), k=k)
  tgt_points = [ tgt[point['id']-1] for point in src_points ]
  demos = []
  for s, t in zip(src_points, tgt_points):
    demo = f"{src_endonym}: {s['sentence']} | {tgt_endonym}: {t['sentence']}"
    demos.append(demo)
  demo_prompt_ids = [2]
  for demo in demos:
    demo_prompt_ids.extend(s_ids(demo))
    demo_prompt_ids.append(2)
  return demo_prompt_ids

In [173]:
demo_prompt_ids = make_demo_prompt(src_endonym, tgt_endonym, src['devtest'], tgt['devtest'], 32)

In [174]:
tokenizer.decode(prompt_ids[-30:])

'rintasyövän kaltaisen taudin eloonjäämisaste voi olla puolet rikkaampien maiden vastaavasta. | English:'

In [175]:
inference_point = src['dev'][1] 
prompt_ids = demo_prompt_ids + s_ids(f"{src_endonym}: {inference_point['sentence']} | {tgt_endonym}:")
prompt_t = torch.LongTensor([prompt_ids]).to(model.device)

In [180]:
with torch.no_grad():
  gen_ids = mt.sample(model, prompt_t, 10, temp=0.3)

In [181]:
tokenizer.decode(gen_ids[0].tolist())

'</s></s></s></s></s></s></s></s></s></s>'

In [98]:
tgt['dev'][1]

{'id': 2,
 'URL': 'https://en.wikinews.org/wiki/Scientists_say_new_medical_diagnostic_chip_can_sort_cells_anywhere_with_an_inkjet',
 'domain': 'wikinews',
 'topic': 'health',
 'has_image': 0,
 'has_hyperlink': 0,
 'sentence': 'Lead researchers say this may bring early detection of cancer, tuberculosis, HIV and malaria to patients in low-income countries, where the survival rates for illnesses such as breast cancer can be half those of richer countries.'}

In [ ]:
tokenizer.decode([2] + gen_ids[0].tolist())

In [52]:
prompt_ids = torch.LongTensor([tokenizer('translate into english, 中文： 谢谢你！').input_ids[:-1]]).to(model.device)

In [39]:
prompt_ids = torch.LongTensor([tokenizer('中文： 谢谢你!').input_ids + [2] + tokenizer('english:').input_ids[1:]]).to(model.device)
                               
                               

In [53]:
prompt_ids.tolist()

[[2, 178680, 1531, 83763, 4, 6, 19773, 13, 6, 13305, 13305, 710]]

In [60]:
with torch.no_grad():
  gen_ids = mt.sample(model, prompt_ids, 200)

In [61]:
tokenizer.decode([2] + gen_ids[0].tolist())

'</s> (English, Chinese)</s> 据说,懂得‘谢谢’这个字的人,都有大智慧。那是因为懂得‘谢谢’的人有聪明的心,即是知足常乐的智慧。那么就会明白每个人来到人间的时候,经过十二玄门的艰难关卡,习得一种深刻的智识,这智识就是:舍得福。</s> 舍得,用舍得的智慧,用也、不也、有、无(4种开关)设,把世间的种地、食材、人、<unk>!生命所需,拿 as;拿 came;拿 take;拿 go、拿 do、拿 make;拿 lie和 etc.加..所以懂得舍得的智慧人,他的心胸开阔,能够洞悉行人的对工夫、思想、心情!用精练的心、深情的眼睛、严肃的态度、轻快的语气、亲切的笑'

In [51]:
print(tokenizer.decode(prompt_ids[0]))

</s> 中文: 谢谢你! | english


In [14]:
inference_point['sentence']

'Stanfordin yliopiston lääketieteen laitoksen tutkijat ilmoittivat maanantaina uuden diagnostiikkatyökalun keksimisestä: solut tyypin mukaan lajitteleva pienenpieni tulostettava siru, joka voidaan valmistaa normaaleilla mustesuihkutulostimilla mahdollisesti noin yhden Yhdysvaltain sentin kappalehintaan.'

In [22]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad

In [12]:
vocab = tokenizer.get_vocab()
tokens = list(sorted(tokenizer.get_vocab(), key=lambda k: vocab[k]))

In [121]:
tokens[:15]

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 ',',
 '.',
 '▁',
 's',
 '-',
 'a',
 '▁de',
 '▁a',
 'e',
 ':',
 'i']